**pCRA Calculator**

The pCRA formula can be found [here](https://sixmanrotation.com/general/cra-and-pcra-finalized). I will not be attempting to modify the existing formula, but the author noted the weakness in the prior distribution is that we should expect someone like Gerrit Cole or Jacob DeGrom to have better strikeout rates and walk rates than other pitchers. For this reason, I look at the previous year statistics and regress it a good amount, but not all the way back towards the mean. This means that our prior distribution will be better because Gerrit Cole will already have a strikeout rate higher than league average.

In [3]:
import pandas as pd
from pybaseball import pitching_stats

In [4]:
# Enter what year you want to calculate pCRA for
year = 2021

In [5]:
df = pitching_stats(year, year)
df2020 = pitching_stats(year - 1,year - 1)
df = pd.merge(df, df2020, on='Name', how='left', suffixes=['_2021', '_2020'])
df.set_index('Name', inplace=True)
df['BK_2021'] = ((df['SO_2021']) + 10.66) / (df['TBF_2021'] + 10.66 + 37.53)
df['BWalk_2021'] = (df['BB_2021'] + 9.58) / (df['TBF_2021'] + 9.58 + 116.2)
df['BBarrels_2021'] = (df['Barrels_2021'] + 13.33) / (df['Events_2021'] + 13.33 + 191.5)

df['pCRA_2021'] = (-8.89*df['BK_2021']) + (8.03*df['BWalk_2021']) + (5.54*df['BBarrels_2021']) + (106*(df['BBarrels_2021']**2)) + 4.55

# 200 innings would be a 50% regression
df['regress_by'] = df['IP_2020'] / 400
df[['SO_2020', 'TBF_2020', 'BB_2020', 'Barrels_2020', 'Events_2020', 'IP_2020', 'regress_by']] = df[['SO_2020', 'TBF_2020', 'BB_2020', 'Barrels_2020', 'Events_2020', 'IP_2020', 'regress_by']].fillna(0)
df['K_reg'] = df['SO_2020'] * df['regress_by'] 
df['BB_reg'] = df['BB_2020'] * df['regress_by'] 
df['TBF_reg'] = df['TBF_2020'] * df['regress_by'] 
df['Barrels_reg'] = df['Barrels_2020'] * df['regress_by'] 
df['Events_reg'] = df['Events_2020'] * df['regress_by'] 

df['BK_2021'] = ((df['SO_2021']) + 10.66*(1-df['regress_by'])+ df['K_reg']) / (df['TBF_2021'] + 48.19*(1-df['regress_by']) + df['TBF_reg'])
df['BWalk_2021'] = (df['BB_2021'] + 9.58*(1-df['regress_by']) + df['BB_reg']) / (df['TBF_2021'] + 125.78*(1-df['regress_by']) + df['TBF_reg'])
df['BBarrels_2021'] = (df['Barrels_2021'] + 13.33*(1-df['regress_by']) + df['Barrels_reg']) / (df['Events_2021'] + 204.83*(1-df['regress_by']) + df['Events_reg'])

df['pCRA_2021_reg'] = (-8.89*df['BK_2021']) + (8.03*df['BWalk_2021']) + (5.54*df['BBarrels_2021']) + (106*(df['BBarrels_2021']**2)) + 4.55
df['pCRA_diff'] = df['pCRA_2021'] - df['pCRA_2021_reg']
df[['pCRA_2021', 'pCRA_2021_reg', 'pCRA_diff']].head(50)

,pCRA_2021,pCRA_2021_reg,pCRA_diff
Name,,,
Jacob deGrom,2.161105,2.198104,-3.699832e-02
Gerrit Cole,2.761373,2.807351,-4.597793e-02
Kevin Gausman,3.150276,3.150276,8.881784e-16
Brandon Woodruff,2.972323,2.954217,1.810577e-02
Zack Wheeler,3.136218,3.193299,-5.708131e-02
Shane Bieber,3.320753,3.216320,1.044330e-01
Nathan Eovaldi,3.789398,3.789398,0.000000e+00
Tyler Glasnow,3.226498,3.226498,4.440892e-16
Clayton Kershaw,3.306515,3.306515,4.440892e-16
